'''
This creates a data frame which contains Image path BB,PHOS,PHOC representation
From IAM handwritten data.

Further it creates files tempText.pickle file which is a dictionary of word and its corresponding PHOSC representation

'''


In [ ]:
#!python train.py --data coco2.yaml --cfg yolov5s.yaml --weights '' --batch-size 8 --epochs 50

In [ ]:
#! python detect.py --source data/datasets/coco2_test/images --weights ./runs/train/exp6/weights/best.pt --conf 0.25


In [ ]:
#! python detect.py --source data/images --weights ./runs/train/exp3/weights/best.pt --conf 0.25


"""
    BELOW PART PARSES WORD AND ITS COORDINATES
"""

In [1]:
def mergeCoordinate(d):
    
    '''
    l=d["cord"]
    
    temp=l[0]
    minx,y1=temp[0],temp[1] 
       
    temp=l[-1]
       
    maxx,y2=temp[2],temp[3]    
    
    return minx,y1,maxx,y2
    '''
    
    minX,minY,maxX,maxY=float('inf'),float('inf'),-float('inf'),-float('inf')  
    
    for indx,l in enumerate(d["cord"]):
        
        minX,minY,maxX,maxY=min(minx,l[0]),min(minx,l[1]),max(maxY,l[2]),max(maxY,l[3])
        
#         if indx==0:
#             constX=l[]
            
#         x,y=l[0],l[-1]
        
#         minx,maxY=min(minx,x),max(maxY,y)
    
    return minX,minY,maxX,maxY

        

In [2]:
#d={'word': 'MOVE', 'cord': [[507, 768, 570, 814], [568, 770, 624, 811], [631, 768, 669, 809], [676, 772, 707, 808], [691, 766, 720, 778]]}
#r=mergeCoordinate(d)
#print("\n\t r:",r)

dumpPath="./data/datasets/forms_1/"

def mergeCoordinate(d):
    
    minX,minY,maxX,maxY=float('inf'),float('inf'),-float('inf'),-float('inf')  
    allCords=[]
    
    #l=d["cord"]
    #l1=l[0]
    #l2=l[-1]
    
    for indx,l in enumerate(d["cord"]):
        
        #print("\n\t l in loop:",l)
        minX,minY,maxX,maxY=min(minX,l[0]),min(minY,l[1]),max(maxX,l[2]),max(maxY,l[3])
        #allCords.append([minX,minY,maxX,maxY])
        #allCords.append(l)
    
    #l=[l1[0],l1[1],l2[-2],l2[-1]]
    l=[minX,minY,maxX,maxY]
    allCords.append(l)
    return allCords#minX,minY,maxX,maxY

def show(img,nm):
    #img.save("./temp/"+nm+".png")

    img=img.resize((1000,800))
    img.show()

#df=["image_name","class","width","height","x1","y1","x2","y2","org_x1","org_y1","org_x2","org_y2"]

'''
    create a dataframe in format xCentre,yCentre,height,width
'''

def fill(indx,img,nm,cropName,fileName,df,x1,y1,x2,y2,currTextVal):
    
    w,h=img.size
    df.loc[indx,"image_name"]=nm
    df.loc[indx,"cropName"]=cropName
    df.loc[indx,"class"]=1
    df.loc[indx,"width"]=w
    df.loc[indx,"height"]=h
    df.loc[indx,"x"]=round((x1+x2)/(2*w),2) 
    df.loc[indx,"y"]=round((y1+y2)/(2*h),2)
    df.loc[indx,"w"]=round(abs(x2-x1)/w,2)
    df.loc[indx,"h"]=round(abs(y2-y1)/h,2)
    df.loc[indx,"org_x1"]=x1
    df.loc[indx,"org_y1"]=y1
    df.loc[indx,"org_x2"]=x2
    df.loc[indx,"org_y2"]=y2
    df.loc[indx,"text"]=currTextVal
    
    xx=str(round((x1+x2)/(2*w),2))
    yy=str(round((y1+y2)/(2*h),2))
    ww=str(round(abs(x2-x1)/w,2))
    hh=str(round(abs(y2-y1)/h,2))
    
    text="0"+" "+xx+" "+yy+" "+ww+" "+hh
    
    with open (dumpPath+fileName,'a') as f: 
        f.write (text+'\n')
    
    return df


'''

    below part parse forms and records coordinate and text 
    
'''

In [ ]:
#[elem.tag for elem in root.iter()]
from collections import defaultdict
import xml.etree.ElementTree as ET
from PIL import Image, ImageDraw
import os
import pandas as pd
import sys
d={}


#imgPath="/home/k/phd/yolov5/data/datasets/formsA-D/a01-000u.png"
imgPath="./data/datasets/forms/"
path="./data/xml//"
#files=["a01-000u.xml","a01-000x.xml"]
allImages=os.listdir(imgPath)
files=os.listdir(path)
df=pd.DataFrame(columns=["image_name","class","width","height","org_x1","org_y1","org_x2","org_y2","text"])
lineNo=0
expCount=0
for indx,nm in enumerate(files):
    
    
    '''
    if indx not in [20,22,32,33,41,46,47,62]:
        continue
        
    if indx>62:
        break
    ''' 
    imgName=nm.split(".xml")[0]+".png"
    if imgName not in allImages:
        continue
    
    if indx%100==0:
        print("\n\t indx:",indx,"\t total:",len(files),"\t\t nm:",nm,"\t unique files:",len(df.image_name.unique()))
        
#     if len(df.image_name.unique())>200:
#         break

    doc = ET.parse(path+nm)
    root = doc.getroot()
    fn=nm.split(".xml")[0]+".txt"

    img=Image.open(imgPath+imgName)
    #img=img.convert('RGB')
    #print("\n\t 1.shape:",img.size)     
    #show(img,"nm")
    #input("check")
#     continue
    img1 = ImageDraw.Draw(img) 

    for node in root.iter('word'):
        #print("\n\t attrib:",node.attrib["text"])
        currTextVal=node.attrib["text"]
        #print("\n\t ",i)
        prevWord=None
        words=defaultdict(list)

        for ele in node.iter():
            #print("\n\t ele:",ele.tag)
            if ele.tag=="word":

                try:
                    if "word" in d.keys():
                        prevWord=d["word"]
                        #print("\n\t prevWord=",d["word"],"\t word:",ele.attrib["text"])
                        
                        #print("\n\t d:",d)
                        allCords=mergeCoordinate(d)
                        #print("\n\t allCords=",allCords)
                        cord=allCords[0]
                        #for cord in allCords:
                        #print("\n\t\t\t cord:",cord)
                        x1,y1,x2,y2=int(cord[0]),int(cord[1]),int(cord[2]),int(cord[3])
                        #area=abs(x2-x1)*abs(y2-y1)
                        #print("\n\t area:",area)
                        #if 1:#abs(x2-x1)*abs(y2-y1)<300000:
                        #img1.line((x1,(y1+y2)/2,x2,(y1+y2)/2),fill="black",width=10)
                        
                        df=fill(lineNo,img,imgName,fn,df,x1,y1,x2,y2,currTextVal)
                        lineNo+=1
                        #print("\n\t img=",img.shape)
                        #img1.rectangle(((x1,y1), (x2,y2)),outline = "red",width=10)
                        #show(img,nm)
                        #input(3)

                except Exception as e:
                    expCount+=1
                    exc_type, exc_obj, exc_tb = sys.exc_info()
                    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
                    #print(exc_type, fname, exc_tb.tb_lineno)
                    #print("\n\t  d exception:",e,"\t line number:",exc_tb.tb_lineno,"\t ",indx)
                    #print("\n\t cord:",cord,"\t exp:",expCount,d)
                    #input("check")

                d={}
                d["word"]=ele.attrib["text"]
                d["cord"]=[]
            elif ele.tag=="cmp":
                #print("\n\t\t current text:",d["word"])
                #print("\n\t\t current co-ordinate:",ele.attrib)
                #print("\n\t\t x:",ele.attrib['x'],"\t y:",ele.attrib['y'])
                x1,y1=int(ele.attrib['x']),int(ele.attrib['y'])
                x2,y2=x1+int(ele.attrib['width']),y1+int(ele.attrib['height'])
                #img1.line((x1,y1,x2,y2),fill="green",width=10)                
                #img1.rectangle(((x1,y1), (x2,y2)), fill="green")

                d["cord"].append([x1,y1,x2,y2])

                allCords=mergeCoordinate(d)
                cord=allCords[0]
                x1,y1,x2,y2=cord[0],cord[1],cord[2],cord[3]
                area=abs(x2-x1)*abs(y2-y1)
                #img1.rectangle(((x1,y1), (x2,y2)),outline = "red",width=3)
                #d={}
                
        if indx%5==0:
            df.to_csv("./data/data3.csv",index=False)

    df.to_csv("./data/data3.csv",index=False)
    #print("\n\t uunique files:",len(df.image_name.unique()))    
    #img.save("./temp/"+nm+".png")
    #img=img.resize((500,400))
    #show(img,imgName)
    #input("press")


In [ ]:
df.columns

'''
    now add phoc and phos label
'''

In [62]:
from phos_label_generator import gen_label
from phoc_label_generator import gen_phoc_label

#generate_phoc_vector("A")

In [64]:
train_word_phos_label = gen_label(list(set(df['text'])))
print("\n\t len:",len(train_word_phos_label))

train_word_phoc_label = gen_phoc_label(list(set(df['text'])))
print("\n\t len:",len(train_word_phoc_label))


KeyError: '2'

In [ ]:
import re

len(list(set(df['text'])))
word=list(set(df['text']))


regex = re.compile('[^a-zA-Z]')
#First parameter is the replacement, second parameter is your input string

tempPhos=[0]*165
tempPhoc=[0.0]*604
#output['Sold Count'] = output['Sold Count'].astype(object)
df["phos"]=df["phos"].astype(object)

for indx,row in df.iterrows():
    
#     if indx>100:
#         break
    
    try:
        #print("\n\t w1:",w)
        w=row["text"]
        w=regex.sub('', w)
        #print("\n\t w2:",w)
        if len(w):
            phos_label = gen_label([w])
            phoc_label = gen_phoc_label([w])
            #print("\n\t train_word_phos_label=",train_word_phos_label)
            #print("\n\t keys:",len(phos_label[w]))
            #print("\n\t type:",type(phos_label[w]))
            #print("\n\t phos_label[w]=",phos_label[w])
            #df.loc[indx,"phoc"]=str(phos_label[w]) gen_phoc_label
            
            #print("\n\t phos_label =",len(phos_label[w].tolist()))
            #print("\n\t phoc_label =",len(phoc_label[w]))

            df.at[indx,"phoc"]=phoc_label[w]
            df.at[indx,"phos"]=phos_label[w]#.tolist()

        else:
            df.at[indx,"phoc"]=tempPhoc
            df.at[indx,"phos"]=tempPhos

        #input("check!!")
        
        
    except Exception as e:
        print("\n\t w:",w,"\t indx:",indx,"\t e:",e)
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\n\t line number:", exc_tb.tb_lineno)
        input("check!!!")
df.to_csv("./data/data3_phosc.csv")
#train_word_phos_label = gen_label(list(set(df['text'])))
#train_word_phos_label = gen_label(word[5])

In [ ]:
import ast
#df.dtypes
phos=df.loc[0,"phoc"]
#phos
#phos[7]
#len(phos)

#phos_label = gen_label([w])
#type(phos_label[w].tolist())
#type(phoc_label[w])
#phos_label[w]#.tolist()

In [ ]:
#!conda info

In [ ]:
!conda install Pillow --y

In [ ]:
import os
os.getcwd()

In [ ]:
import os
from PIL import Image
path="/home/k/phd/yolov5/runs/detect/exp7/"
path1="/home/k/phd/yolov5/runs/detect/compress/"

imgList=os.listdir(path)

for indx,nm in enumerate(imgList):
    
    img=Image.open(path+nm)
    img=img.resize((500,400))
    
    img.save(path1+nm)



In [ ]:
'''
    text file creation for YOLO
'''

In [ ]:
# Python program to illustrate
# Append vs write mode
file1 = open("myfile.txt", "w")
L = ["This is Delhi \n", "This is Paris \n", "This is London"]
file1.writelines(L)
file1.close()

In [7]:

import pandas as pd
df=pd.read_csv("./data/data3_phosc.csv")

unqImages=set(data.image_name)
print("\n\t unique:",len(unqImages))
#print(data.columns)


	 unique: 1539


In [ ]:
os.getcwd()
data.dtypes

In [9]:
#for file in ['train','test']:
import numpy as np
import os
'''
allImages=os.listdir("/home/k/phd/yolov5/data/datasets/forms/")
images_num=len(os.listdir("/home/k/phd/yolov5/data/datasets/forms/"))
images_path = "/home/k/phd/yolov5/data/datasets/forms/"#os.getcwd()+f"/mnist_{file}"
labels_txt = os.getcwd()+"/"+"mnist_train.txt"
labels_txt2 = os.getcwd()+"/"+"mnist_test.txt"
'''

allImages=os.listdir("/home/k/phd/yolov5/data/datasets/forms/")
images_num=len(os.listdir("/home/k/phd/yolov5/data/datasets/forms/"))
images_path = "/home/k/phd/yolov5/data/datasets/forms/"#os.getcwd()+f"/mnist_{file}"
labels_txt = os.getcwd()+"/"+"mnist_train1.txt"
labels_txt2 = os.getcwd()+"/"+"mnist_test1.txt"

lastName=""
curLine=""
prevLine=""
unqImg=[]
exceptionCount=0

for indx,info in df.iterrows():
    try:
        if indx%10==0:
            print("\n\t unq images:",len(unqImg))

        if len(unqImg)<500:

            with open(labels_txt, "a") as wf:
                image_path =images_path+info["image_name"]

                if indx%100==0:
                    print("\n\t path:",image_path)

                if info["image_name"]!=lastName and info["image_name"] not in unqImg:
                    curLine=image_path

                    if len(prevLine):
                        #print("\n\t prevLine:",prevLine)
                        #wf.write( image_path+ "\n")
                        wf.write(prevLine+"\n")

                else:
                    xmin,ymin=str(int(info["org_x1"])),str(int(info["org_y1"]))
                    xmax,ymax=str(int(info["org_x2"])),str(int(info["org_y2"]))
                    curLine += ' ' + ','.join([xmin, ymin, xmax, ymax, str(0)])
                    prevLine=curLine

                lastName=info["image_name"]

        elif 500<len(unqImg)<600:

            with open(labels_txt2, "a") as wf:
                image_path =images_path+info["image_name"]

                if indx%100==0:
                    print("\n\t path:",image_path)

                if info["image_name"]!=lastName:
                    curLine=image_path

                    if len(prevLine):
                        #print("\n\t prevLine:",prevLine)
                        #wf.write( image_path+ "\n")
                        wf.write(prevLine+"\n")

                else:
                    xmin,ymin=str(int(info["org_x1"])),str(int(info["org_y1"]))
                    xmax,ymax=str(int(info["org_x2"])),str(int(info["org_y2"]))
                    curLine += ' ' + ','.join([xmin, ymin, xmax, ymax, str(0)])
                    prevLine=curLine

                lastName=info["image_name"]
        elif 600<len(unqImg):
            break
            
        if info["image_name"] not in unqImg:
            unqImg.append(info["image_name"])

    except Exception as e:
        exceptionCount+=1
        print("\n\t exception index:",indx,"\t count:",exceptionCount)



	 unq images: 0

	 path: /home/k/phd/yolov5/data/datasets/forms/l04-087.png

	 unq images: 1

	 unq images: 1

	 unq images: 1

	 unq images: 1

	 unq images: 1

	 unq images: 2

	 unq images: 2

	 unq images: 2

	 unq images: 2

	 unq images: 2

	 path: /home/k/phd/yolov5/data/datasets/forms/d01-024.png

	 unq images: 2

	 unq images: 2

	 unq images: 3

	 unq images: 3

	 unq images: 3

	 unq images: 3

	 unq images: 3

	 unq images: 3

	 unq images: 3

	 unq images: 4

	 path: /home/k/phd/yolov5/data/datasets/forms/k04-022.png

	 unq images: 4

	 unq images: 4

	 unq images: 4

	 unq images: 4

	 unq images: 4

	 unq images: 4

	 unq images: 4

	 unq images: 5

	 unq images: 5

	 unq images: 5

	 path: /home/k/phd/yolov5/data/datasets/forms/c02-000.png

	 unq images: 5

	 unq images: 5

	 unq images: 5

	 unq images: 5

	 unq images: 5

	 unq images: 6

	 unq images: 6

	 unq images: 6

	 unq images: 6

	 unq images: 6

	 path: /home/k/phd/yolov5/data/datasets/forms/g06-026k.png

	


	 unq images: 49

	 unq images: 49

	 path: /home/k/phd/yolov5/data/datasets/forms/h07-037.png

	 unq images: 50

	 unq images: 50

	 unq images: 50

	 unq images: 50

	 unq images: 50

	 unq images: 50

	 unq images: 50

	 unq images: 50

	 unq images: 50

	 unq images: 51

	 path: /home/k/phd/yolov5/data/datasets/forms/r02-013.png

	 unq images: 51

	 unq images: 51

	 unq images: 51

	 unq images: 51

	 unq images: 51

	 unq images: 51

	 unq images: 51

	 unq images: 52

	 unq images: 52

	 unq images: 52

	 path: /home/k/phd/yolov5/data/datasets/forms/c06-005.png

	 unq images: 52

	 unq images: 52

	 unq images: 52

	 unq images: 52

	 unq images: 53

	 unq images: 53

	 unq images: 53

	 unq images: 53

	 unq images: 53

	 unq images: 53

	 path: /home/k/phd/yolov5/data/datasets/forms/b04-020.png

	 unq images: 53

	 unq images: 54

	 unq images: 54

	 unq images: 54

	 unq images: 54

	 unq images: 54

	 unq images: 54

	 unq images: 54

	 unq images: 55

	 unq images: 55

	 p


	 unq images: 102

	 unq images: 103

	 unq images: 103

	 unq images: 103

	 unq images: 103

	 path: /home/k/phd/yolov5/data/datasets/forms/r06-103.png

	 unq images: 103

	 unq images: 103

	 unq images: 104

	 unq images: 104

	 unq images: 104

	 unq images: 104

	 unq images: 104

	 unq images: 104

	 unq images: 104

	 unq images: 104

	 path: /home/k/phd/yolov5/data/datasets/forms/h01-014.png

	 unq images: 104

	 unq images: 105

	 unq images: 105

	 unq images: 105

	 unq images: 105

	 unq images: 105

	 unq images: 105

	 unq images: 105

	 unq images: 106

	 unq images: 106

	 path: /home/k/phd/yolov5/data/datasets/forms/h07-080.png

	 unq images: 106

	 unq images: 106

	 unq images: 106

	 unq images: 106

	 unq images: 106

	 unq images: 107

	 unq images: 107

	 unq images: 107

	 unq images: 107

	 unq images: 107

	 path: /home/k/phd/yolov5/data/datasets/forms/c04-139.png

	 unq images: 107

	 unq images: 107

	 unq images: 107

	 unq images: 108

	 unq images: 108



	 unq images: 149

	 unq images: 149

	 unq images: 149

	 unq images: 149

	 unq images: 149

	 unq images: 149

	 unq images: 150

	 path: /home/k/phd/yolov5/data/datasets/forms/f02-030.png

	 unq images: 150

	 unq images: 150

	 unq images: 150

	 unq images: 150

	 unq images: 150

	 unq images: 151

	 unq images: 151

	 unq images: 151

	 unq images: 151

	 unq images: 151

	 path: /home/k/phd/yolov5/data/datasets/forms/g06-045j.png

	 unq images: 151

	 unq images: 151

	 unq images: 152

	 unq images: 152

	 unq images: 152

	 unq images: 152

	 unq images: 152

	 unq images: 152

	 unq images: 152

	 unq images: 153

	 path: /home/k/phd/yolov5/data/datasets/forms/g06-045d.png

	 unq images: 153

	 unq images: 153

	 unq images: 153

	 unq images: 153

	 unq images: 153

	 unq images: 153

	 unq images: 154

	 unq images: 154

	 unq images: 154

	 unq images: 154

	 path: /home/k/phd/yolov5/data/datasets/forms/b04-034.png

	 unq images: 154

	 unq images: 154

	 unq images: 15


	 unq images: 202

	 unq images: 202

	 unq images: 202

	 unq images: 203

	 unq images: 203

	 unq images: 203

	 unq images: 203

	 path: /home/k/phd/yolov5/data/datasets/forms/m02-090.png

	 unq images: 203

	 unq images: 203

	 unq images: 203

	 unq images: 203

	 unq images: 203

	 unq images: 203

	 unq images: 203

	 unq images: 203

	 unq images: 204

	 unq images: 204

	 path: /home/k/phd/yolov5/data/datasets/forms/p06-104.png

	 unq images: 204

	 unq images: 204

	 unq images: 204

	 unq images: 204

	 unq images: 204

	 unq images: 205

	 unq images: 205

	 unq images: 205

	 unq images: 205

	 unq images: 205

	 path: /home/k/phd/yolov5/data/datasets/forms/f07-042b.png

	 unq images: 205

	 unq images: 205

	 unq images: 206

	 unq images: 206

	 unq images: 206

	 unq images: 206

	 unq images: 206

	 unq images: 206

	 unq images: 206

	 unq images: 206

	 path: /home/k/phd/yolov5/data/datasets/forms/h01-024.png

	 unq images: 207

	 unq images: 207

	 unq images: 207

...


	 unq images: 349

	 unq images: 349

	 unq images: 350

	 path: /home/k/phd/yolov5/data/datasets/forms/r06-011.png

	 unq images: 350

	 unq images: 350

	 unq images: 350

	 unq images: 350

	 unq images: 350

	 unq images: 350

	 unq images: 350

	 unq images: 351

	 unq images: 351

	 unq images: 351

	 path: /home/k/phd/yolov5/data/datasets/forms/p03-135.png

	 unq images: 351

	 unq images: 351

	 unq images: 351

	 unq images: 351

	 unq images: 351

	 unq images: 352

	 unq images: 352

	 unq images: 352

	 unq images: 352

	 unq images: 352

	 path: /home/k/phd/yolov5/data/datasets/forms/m03-095.png

	 unq images: 352

	 unq images: 353

	 unq images: 353

	 unq images: 353

	 unq images: 354

	 unq images: 354

	 unq images: 354

	 unq images: 354

	 unq images: 354

	 unq images: 354

	 path: /home/k/phd/yolov5/data/datasets/forms/l07-135.png

	 unq images: 354

	 unq images: 355

	 unq images: 355

	 unq images: 355

	 unq images: 355

	 unq images: 355

	 unq images: 355



	 unq images: 400

	 unq images: 401

	 unq images: 401

	 unq images: 401

	 unq images: 401

	 path: /home/k/phd/yolov5/data/datasets/forms/b04-181.png

	 unq images: 401

	 unq images: 401

	 unq images: 402

	 unq images: 402

	 unq images: 402

	 unq images: 402

	 unq images: 403

	 unq images: 403

	 unq images: 403

	 unq images: 403

	 path: /home/k/phd/yolov5/data/datasets/forms/c02-078.png

	 unq images: 403

	 unq images: 403

	 unq images: 403

	 unq images: 403

	 unq images: 403

	 unq images: 403

	 unq images: 404

	 unq images: 404

	 unq images: 404

	 unq images: 404

	 path: /home/k/phd/yolov5/data/datasets/forms/a02-042.png

	 unq images: 404

	 unq images: 404

	 unq images: 404

	 unq images: 405

	 unq images: 405

	 unq images: 405

	 unq images: 405

	 unq images: 405

	 unq images: 405

	 unq images: 405

	 path: /home/k/phd/yolov5/data/datasets/forms/a05-089.png

	 unq images: 405

	 unq images: 405

	 unq images: 406

	 unq images: 406

	 unq images: 406



	 unq images: 450

	 unq images: 450

	 path: /home/k/phd/yolov5/data/datasets/forms/e02-094.png

	 unq images: 451

	 unq images: 451

	 unq images: 451

	 unq images: 451

	 unq images: 451

	 unq images: 451

	 unq images: 451

	 unq images: 451

	 unq images: 452

	 unq images: 452

	 path: /home/k/phd/yolov5/data/datasets/forms/g06-026a.png

	 unq images: 452

	 unq images: 452

	 unq images: 452

	 unq images: 452

	 unq images: 453

	 unq images: 453

	 unq images: 453

	 unq images: 453

	 unq images: 453

	 unq images: 453

	 path: /home/k/phd/yolov5/data/datasets/forms/r03-110.png

	 unq images: 453

	 unq images: 453

	 unq images: 453

	 unq images: 453

	 unq images: 453

	 unq images: 454

	 unq images: 454

	 unq images: 454

	 unq images: 454

	 unq images: 454

	 path: /home/k/phd/yolov5/data/datasets/forms/k02-117.png

	 unq images: 454

	 unq images: 454

	 unq images: 454

	 unq images: 454

	 unq images: 455

	 unq images: 455

	 unq images: 455

	 unq images: 455

	 unq images: 503

	 unq images: 504

	 unq images: 504

	 unq images: 504

	 unq images: 504

	 unq images: 504

	 path: /home/k/phd/yolov5/data/datasets/forms/h07-066a.png

	 unq images: 504

	 unq images: 505

	 unq images: 505

	 unq images: 505

	 unq images: 505

	 unq images: 505

	 unq images: 505

	 unq images: 505

	 unq images: 505

	 unq images: 505

	 path: /home/k/phd/yolov5/data/datasets/forms/e04-062.png

	 unq images: 506

	 unq images: 506

	 unq images: 506

	 unq images: 506

	 unq images: 506

	 unq images: 506

	 unq images: 506

	 unq images: 506

	 unq images: 506

	 unq images: 506

	 path: /home/k/phd/yolov5/data/datasets/forms/p03-057.png

	 unq images: 507

	 unq images: 507

	 unq images: 507

	 unq images: 507

	 unq images: 507

	 unq images: 507

	 unq images: 507

	 unq images: 507

	 unq images: 507

	 unq images: 508

	 path: /home/k/phd/yolov5/data/datasets/forms/a06-020.png

	 unq images: 508

	 unq images: 508

	 unq images: 508

	 unq images: 508



	 unq images: 550

	 unq images: 551

	 unq images: 551

	 path: /home/k/phd/yolov5/data/datasets/forms/m04-107.png

	 unq images: 551

	 unq images: 551

	 unq images: 551

	 unq images: 551

	 unq images: 551

	 unq images: 552

	 unq images: 552

	 unq images: 552

	 unq images: 552

	 unq images: 552

	 path: /home/k/phd/yolov5/data/datasets/forms/m03-033.png

	 unq images: 552

	 unq images: 552

	 unq images: 553

	 unq images: 553

	 unq images: 553

	 unq images: 553

	 unq images: 553

	 unq images: 553

	 unq images: 553

	 unq images: 553

	 path: /home/k/phd/yolov5/data/datasets/forms/g06-031p.png

	 unq images: 554

	 unq images: 554

	 unq images: 554

	 unq images: 554

	 unq images: 554

	 unq images: 554

	 unq images: 554

	 unq images: 555

	 unq images: 555

	 unq images: 555

	 path: /home/k/phd/yolov5/data/datasets/forms/n04-044.png

	 unq images: 555

	 unq images: 555

	 unq images: 555

	 unq images: 555

	 unq images: 555

	 unq images: 555

	 unq images: 556

In [3]:
'''
    below part convert dataframe to json 
'''
import pandas as pd
df=pd.read_csv("/home/k/phd/TensorFlow-2.x-YOLOv3/data/data3_phosc.csv")
print("\n\t df:",df.shape)
#df.columns


	 df: (115187, 16)


In [93]:
import collections
import json
import numpy as np
import re

d=collections.defaultdict(dict)
tempDir={}
tempCord={}
tempText={}
regex = re.compile('[^a-zA-Z]')

for indx,info in df.iterrows():
    
    tempDir2={}
    #print("\n\t name:",info["image_name"])
    temp=[info['org_x1'],info['org_y1'],info['org_x2'],info['org_y2']]
    '''
    tempDir2["text"]=info['text']
    ''' 
    w=info['text']
    w=regex.sub('', w)
    #print("\n\t w2:",w)
    
    if len(w):

        tempDir2["phoc"]=np.asarray(gen_phoc_label([w])[w]).astype(np.float32)
        tempDir2["phos"]=np.asarray(gen_label([w])[w]).astype(np.float32)
        tempText[info['text']]=tempDir2
    else:
        
        tempDir2["phoc"]=np.zeros(604)
        tempDir2["phos"]=np.zeros(165)
        tempText[info['text']]=tempDir2
    
    
'''
#gen_phoc_label([info['text']])[info['text']]
#gen_label([info['text']])[info['text']]
'''
#tempText.keys()

"\n#gen_phoc_label([info['text']])[info['text']]\n#gen_label([info['text']])[info['text']]\n"

In [94]:
#for ele in info['phos']
#vec=gen_label(["remained"])['remained']
#tempDict={}
#tempDict["remained"]=vec
#type(tempDict["remained"])

# with open("./data/delMe.json", "w") as outfile:
#     json.dump(tempDict, outfile)

# f = open("./data/delMe.json")
# tempDict2= json.load(f)
# type(tempDict2["remained"])


import pickle


with open('filename.pickle', 'wb') as handle:
    pickle.dump(tempText, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('filename.pickle', 'rb') as handle:
    b = pickle.load(handle)

#print (tempText == b)
'''
print("1",type(b['it']))
print("2",type(tempText['it']))
print(b.keys())
print(tempText.keys())
print("1.",b)
'''
#print("2.",tempText)
#dict_keys(['it', 'his', 'imagination', 'or', 'was', 'the', 'panel'])

'\nprint("1",type(b[\'it\']))\nprint("2",type(tempText[\'it\']))\nprint(b.keys())\nprint(tempText.keys())\nprint("1.",b)\n'

In [92]:
t=list(b.keys())[0]
len(b[t]['phos'])

165

In [8]:


f = open("./data/tempCord.json")
tempCord1= json.load(f)

f = open("./data/tempText.json")
tempText1 = json.load(f)



'''
with open("./data/dataframe.json", "w") as outfile:
    json.dump(d, outfile)
'''


'\nwith open("./data/dataframe.json", "w") as outfile:\n    json.dump(d, outfile)\n'

In [80]:
#gen_phoc_label([info['text']])[info['text']]
#gen_label([info['text']])[info['text']]
#tempDir2.keys()
#tempText
#type(b['it']['phos'])
tempText['it']['phoc'][0]

0

'''
    counting words in 500 training examples data required to Sir
'''

In [16]:
#for file in ['train','test']:
import numpy as np
import os
from collections import defaultdict
allImages=os.listdir("/home/k/phd/yolov5/data/datasets/forms/")
images_num=len(os.listdir("/home/k/phd/yolov5/data/datasets/forms/"))
images_path = "/home/k/phd/yolov5/data/datasets/forms/"#os.getcwd()+f"/mnist_{file}"
labels_txt = os.getcwd()+"/"+"mnist_train1.txt"
labels_txt2 = os.getcwd()+"/"+"mnist_test1.txt"

lastName=""
curLine=""
prevLine=""
unqImg=[]
exceptionCount=0
unqWords=defaultdict(int)
totTrainWord=0
unqWordsTest=defaultdict(int)
totTestWords=0
totWords=0

for indx,info in df.iterrows():
    text=info['text']
    try:
        if indx%1000==0:
            print("\n\t unq images:",len(unqImg))

        if len(unqImg)<500:
            unqWords[text]+=1
            totTrainWord+=1
        elif 500<len(unqImg)<600:
            unqWordsTest[text]+=1
            totTestWords+=1
        elif 600<len(unqImg):
            break
            
        if info["image_name"] not in unqImg:
            unqImg.append(info["image_name"])
        
        totWords+=1                    

    except Exception as e:
        exceptionCount+=1
        print("\n\t exception index:",indx,"\t count:",exceptionCount,"\t e:",e)



	 unq images: 0

	 unq images: 14

	 unq images: 27

	 unq images: 40

	 unq images: 53

	 unq images: 66

	 unq images: 79

	 unq images: 92

	 unq images: 106

	 unq images: 119

	 unq images: 133

	 unq images: 146

	 unq images: 160

	 unq images: 173

	 unq images: 185

	 unq images: 199

	 unq images: 212

	 unq images: 225

	 unq images: 238

	 unq images: 250

	 unq images: 263

	 unq images: 276

	 unq images: 290

	 unq images: 303

	 unq images: 317

	 unq images: 331

	 unq images: 344

	 unq images: 358

	 unq images: 371

	 unq images: 385

	 unq images: 400

	 unq images: 414

	 unq images: 427

	 unq images: 440

	 unq images: 453

	 unq images: 467

	 unq images: 480

	 unq images: 493

	 unq images: 506

	 unq images: 520

	 unq images: 534

	 unq images: 547

	 unq images: 560

	 unq images: 573

	 unq images: 586

	 unq images: 600


In [ ]:
=defaultdict(int)
totTrainWord=0
unqWordsTest=defaultdict(int)
totTestWords=0
totWords=0


In [17]:
print("\n\t total train  unique words:",len(unqWords.keys()))
print("\n\t total test unique words",len(unqWordsTest.keys()))
print("\n\t train words:",totTrainWord,"\t test words:",totTestWords,"\t total train+test:",totWords)


	 total train  unique words: 7072

	 total test unique words 2428

	 train words: 37471 	 test words: 7436 	 total train+test: 45041


In [18]:
trainW=set(unqWords.keys())
testW=set(unqWordsTest.keys())
common=list(trainW.intersection(testW))

print("\n\t como:",len(common))


	 como: 1586


In [6]:
df.head()

,Unnamed: 0,image_name,class,width,height,org_x1,org_y1,org_x2,org_y2,text,x,y,w,h,phoc,phos
0,0,l04-087.png,1,2479,3542,361,587,480,652,it,0.17,0.17,0.05,0.02,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 1. 0. 0. 0. 0. ...
1,1,l04-087.png,1,2479,3542,507,600,550,648,his,0.21,0.18,0.02,0.01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[1. 0. 1. 0. 1. 0. 0. 2. 0. 0. 0. 1. 0. 0. 0. ...
2,2,l04-087.png,1,2479,3542,603,603,689,652,imagination,0.26,0.18,0.03,0.01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",[ 0. 1. 4. 0. 1. 0. 4. 10. 0. 0. 1. ...
3,3,l04-087.png,1,2479,3542,768,593,1120,686,or,0.38,0.18,0.14,0.03,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. ...
4,4,l04-087.png,1,2479,3542,1164,620,1209,648,was,0.48,0.18,0.02,0.01,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0. 0. 2. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. ...
